This is the full flow of Data pipeline:
1. Download stock data, news data from Alpaca. Datatime are rounded to next day if the news is published late.
3. Aggregate crawled Reuters' news data.
4. Process Reuters news data datetime value to match trading date.

# 1. Fetch price data from Yfinance api

In [ ]:
!pip install yfinance

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [ ]:
from typing import List
import pandas as pd
import yfinance as yf

def download_data(start_day: str, end_day: str, tickers: List[str]) -> List[pd.DataFrame]:
    """Downloads adjusted close price data for given tickers within a date range."""
    df_list = []
    for ticker in tickers:
        print(f'Downloading data for {ticker}')
        # Set auto_adjust to False to include the 'Adj Close' column
        data = yf.download(ticker, start=start_day, end=end_day)
        data = data.reset_index()
        data['Date'] = data['Date'].dt.date
        data = data[['Date', 'Close']]
        data = data.rename(columns={'Date': 'date', 'Close': ticker})
        df_list.append(data)
    return df_list

In [ ]:
# prompt: Mount gg drive and create  /content/drive/My Drive/price_data path
from google.colab import drive
import os

drive.mount('/content/drive')

# Create the path if it doesn't exist
price_data_path = '/content/drive/My Drive/NCKH/price_data'
if not os.path.exists(price_data_path):
  os.makedirs(price_data_path)
  print(f"Created directory: {price_data_path}")
else:
  print(f"Directory already exists: {price_data_path}")


Mounted at /content/drive
Directory already exists: /content/drive/My Drive/NCKH/price_data


In [ ]:
start_day = '2023-08-17'
end_day = '2025-04-10'
tickers = ["TSLA", "AMZN", "MSFT", "NFLX"]

df_list = download_data(start_day, end_day, tickers)


/tmp/ipython-input-18-1687276188.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_day, end=end_day)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-18-1687276188.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_day, end=end_day)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-18-1687276188.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_day, end=end_day)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-18-1687276188.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_day, end=end_day)
[*********************100%***********************]  1 of 1 completed

In [ ]:
for df in df_list:
  print(df.columns)

MultiIndex([('date',     ''),
            ('TSLA', 'TSLA')],
           names=['Price', 'Ticker'])
MultiIndex([('date',     ''),
            ('AMZN', 'AMZN')],
           names=['Price', 'Ticker'])
MultiIndex([('date',     ''),
            ('MSFT', 'MSFT')],
           names=['Price', 'Ticker'])
MultiIndex([('date',     ''),
            ('NFLX', 'NFLX')],
           names=['Price', 'Ticker'])


In [ ]:
import pickle

def combine_dataframes(df_list: List[pd.DataFrame], path: str, tickers: List[str]) -> dict:
    """Combines dataframes of different tickers into a single dictionary and saves it as a pickle file."""
    df_dicts = [dict(zip(df[('date', '')], df[(ticker, ticker)])) for df, ticker in zip(df_list, tickers)]
    combined_dict = {date: {'price': {}} for df_dict in df_dicts for date in df_dict}
    for i, df_dict in enumerate(df_dicts):
        for date, price in df_dict.items():
            combined_dict[date]['price'][tickers[i]] = price
    pkl_filename = path + 'price.pkl'
    with open(pkl_filename, 'wb') as file:
        pickle.dump(combined_dict, file)
    print(f'Price data saved to: {pkl_filename}')
    return combined_dict

In [ ]:
import pandas as pd
import os # Import the os module

price_path = '/content/drive/My Drive/NCKH/price_data/' # Corrected path
data = pd.read_pickle(os.path.join(price_path, "price_data.pkl")) # Use os.path.join

# Print the first 5 items of the dictionary
for item in list(data.items())[:5]:
  print(item)

(
    datetime.date(2023, 8, 17),
    {'price': {'TSLA': 219.22000122070312, 'AMZN': 133.97999572753906, 'MSFT': 312.6887512207031, 'NFLX': 403.0}}
)

(
    datetime.date(2023, 8, 18),
    {
        'price': {
            'TSLA': 215.49000549316406,
            'AMZN': 133.22000122070312,
            'MSFT': 312.2939758300781,
            'NFLX': 404.5299987792969
        }
    }
)

(
    datetime.date(2023, 8, 21),
    {
        'price': {
            'TSLA': 231.27999877929688,
            'AMZN': 134.67999267578125,
            'MSFT': 317.62261962890625,
            'NFLX': 408.2900085449219
        }
    }
)

(
    datetime.date(2023, 8, 22),
    {
        'price': {
            'TSLA': 233.19000244140625,
            'AMZN': 134.25,
            'MSFT': 318.19488525390625,
            'NFLX': 413.1700134277344
        }
    }
)

(
    datetime.date(2023, 8, 23),
    {
        'price': {
            'TSLA': 236.86000061035156,
            'AMZN': 135.52000427246094,
            'MSFT': 322.67486572265625,
            'NFLX': 427.54998779296875
        }
    }
)

In [ ]:
os.path.exists("/content/drive/My Drive/NCKH/price_data/")

False

# 2. Fetch data from Alpaca Api.


##a. Alpaca document instruction


This is data fetching instruction provided by Alpaca documentation. I do this as an example to understand the retrieved data structure.

In [ ]:
import requests

url = "https://data.alpaca.markets/v1beta1/news"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PK900CRTGLIE97VJER4F",
    "APCA-API-SECRET-KEY": "PWkMvZCTT0ajJcX3O7xSCxDOyCnxaXUaCPQKuHTS"
}

params = {
    'start': '2024-08-17',
    'end': '2024-08-20',
    'sort': 'asc',
    'symbols': 'TSLA,NFLX,MSFT,AMZN,COIN',
    'limit': 50,
    'include_content': True,
    'exclude_contentless': True
}

response = requests.get(url, params = params, headers=headers)

In [ ]:
import json

# Parse the JSON string in response.text
data = json.loads(response.text)

# Print the keys of the main dictionary
print("Keys in the response:")
print(data.keys())

print("Total retrieved news:")
print(len(data['news']))

# If the main structure is a dictionary and contains a list (like 'news'),
# you can inspect the structure of an item in the list.
if 'news' in data and isinstance(data['news'], list) and len(data['news']) > 0:
    print("\nKeys in the first news item:")
    print(data['news'][0].keys())

    print("\nSample of the first news item:")
    display(data['news'][0])
elif isinstance(data, list) and len(data) > 0:
    print("\nKeys in the first item of the list response:")
    print(data[0].keys())

    print("\nSample of the first item:")
    display(data[0])

Keys in the response:
dict_keys(['news', 'next_page_token'])
Total retrieved news:
27

Keys in the first news item:
dict_keys(['author', 'content', 'created_at', 'headline', 'id', 'images', 'source', 'summary', 'symbols', 'updated_at', 'url'])

Sample of the first news item:


{'author': 'Anan Ashraf',
 'content': '<p>EV giant <strong>Tesla Inc.</strong> (NASDAQ:<a class="ticker" href="https://www.benzinga.com/stock/TSLA#NASDAQ">TSLA</a>) has a massive edge on the bridge between <a href="https://www.benzinga.com/news/24/02/36946524/investor-presentation-on-elon-musks-xai-reportedly-flexes-muskonomy-draws-parallels-with-openais-tra">AI and the physical world</a>, thanks to its factories, <strong>Elon Musk</strong>&#8216;s biographer <strong>Ashlee Vance </strong>said on Friday, citing an unnamed AI expert.</p>\n\n\n\n<p><strong>What Happened:</strong> Vance took to X to detail the several ways in which Tesla has an edge over rivals owing to Musk&#8217;s portfolio of companies and several factories. Vance said that the opinion belonged to an individual with &#8220;very deep knowledge of AI and modern factories.&#8221; The biographer, however, did not name the person, saying that he is &#8220;not allowed&#8221; to do so.</p>\n\n\n\n<p>&#8220;I think none of the

"Next_page_token" is none means that we reach the end of the page.

In [ ]:
next_page_token = data['next_page_token']
print(next_page_token)

None


##b. Fetching Alpaca news for project


In [ ]:
#dependencies
import os
import time
import shutil
import httpx
import tenacity
import polars as pl
from rich import print
from tqdm import tqdm
from uuid import uuid4
from datetime import date, timedelta, datetime
from typing import List, Dict, Tuple, Union
from tenacity import retry, stop_after_attempt, wait_fixed

In [ ]:
END_POINT_TEMPLATE = "https://data.alpaca.markets/v1beta1/news?start={start_date}&end={end_date}&limit=50&symbols={symbol}"
END_POINT_TEMPLATE_LINK_PAGE = "https://data.alpaca.markets/v1beta1/news?limit=50&symbol={symbol}&page_token={page_token}"
NUM_NEWS_PER_RECORD = 200
MAX_ATTEMPTS = 5
WAIT_TIME = 60
MAX_WORKERS = 30

In [ ]:
def round_to_next_day(date: pl.Expr) -> pl.Expr:
    # Check the condition for rolling over to the next day
    condition = (date.dt.hour() >= 16) & ((date.dt.minute() > 0) | (date.dt.second() > 0))

    # Use when().then().otherwise() with offset_by to correctly handle date rollovers
    return pl.when(condition).then(date.dt.offset_by("1d")).otherwise(date)

In [ ]:
class ScraperError(Exception):
    pass

In [ ]:
class RecordContainerFull(Exception):
    pass

In [ ]:
class ParseRecordContainer:
    def __init__(self, symbol: str) -> None:
        self.symbol = symbol
        self.record_counter = 0
        self.author_list = []
        self.content_list = []
        self.date_list = []
        self.source_list = []
        self.summary_list = []
        self.title_list = []
        self.url_list = []

    def add_records(self, records: List[Dict[str, str]]) -> None:
        for cur_record in records:
            self.author_list.append(cur_record["author"])
            self.content_list.append(cur_record["content"])
            date = cur_record["created_at"].rstrip("Z")
            self.date_list.append(datetime.fromisoformat(date))
            self.source_list.append(cur_record["source"])
            self.summary_list.append(cur_record["summary"])
            self.title_list.append(cur_record["headline"])
            self.url_list.append(cur_record["url"])
            self.record_counter += 1
            if self.record_counter == NUM_NEWS_PER_RECORD:
                raise RecordContainerFull

    def pop(self, align_next_date: bool = True) -> Union[pl.DataFrame, None]:
        if self.record_counter == 0:
            return None
        return_df = pl.DataFrame(
            {
                "author": self.author_list,
                "content": self.content_list,
                "datetime": self.date_list,
                "source": self.source_list,
                "summary": self.summary_list,
                "title": self.title_list,
                "url": self.url_list,
            }
        )
        if align_next_date:
            return_df = return_df.with_columns(
                round_to_next_day(return_df["datetime"]).alias("date"),
            )
        else:
            return_df = return_df.with_columns(
                pl.col("datetime").date().alias("date"),
            )
        return return_df.with_columns(pl.lit(self.symbol).alias("equity"))

In [ ]:
@retry(stop=stop_after_attempt(MAX_ATTEMPTS), wait=wait_fixed(WAIT_TIME))
def query_one_record(args: Tuple[date, str], temp_dir: str, include_content: bool = False, exclude_contentless = False) -> None:
    date, symbol = args
    next_date = date + timedelta(days=1)
    request_header = {
    "APCA-API-KEY-ID": "PK900CRTGLIE97VJER4F",
    "APCA-API-SECRET-KEY": "PWkMvZCTT0ajJcX3O7xSCxDOyCnxaXUaCPQKuHTS",
    }
    container = ParseRecordContainer(symbol)
    with httpx.Client() as client:
        # first request
        url = END_POINT_TEMPLATE.format(
            start_date=date.strftime("%Y-%m-%d"),
            end_date=next_date.strftime("%Y-%m-%d"),
            symbol=symbol,
        )
        if include_content:
             url += "&include_content=True"
        if exclude_contentless:
             url += "&exclude_contentless=True"

        response = client.get(
            url,
            headers=request_header,
        )
        if response.status_code != 200:
            print("[red]Hit limit[/red]")
            raise ScraperError(response.text)
        result = response.json()
        next_page_token = result["next_page_token"]
        container.add_records(result["news"])

        while next_page_token:
            try:
                url = END_POINT_TEMPLATE_LINK_PAGE.format(
                    symbol=symbol, page_token=next_page_token
                )
                if include_content:
                     url += "&include_content=True"
                if exclude_contentless:
                     url += "&exclude_contentless=True"

                response = client.get(
                    url,
                    headers=request_header,
                )
                if response.status_code != 200:
                    raise ScraperError(response.text)
                result = response.json()
                next_page_token = result["next_page_token"]
                container.add_records(result["news"])
            except RecordContainerFull:
                break

    result = container.pop(align_next_date=True)
    if result is not None:
        result.write_parquet(os.path.join(temp_dir, f"{uuid4()}.parquet"))

In [ ]:
def main_sync() -> None:
    # load data
    import pandas as pd
    # Assuming the price data is already in the correct path as previously defined
    price_data_path = r'/content/drive/My Drive/NCKH/price_data/price_data.pkl' # Using the previously defined price_path structure
    with open(price_data_path, 'rb') as f:
        combined_price_data = pickle.load(f)

    # Convert the dictionary data to a list of dictionaries suitable for Polars
    price_records = []
    for date, data in combined_price_data.items():
        if 'price' in data:
            for ticker, price in data['price'].items():
                price_records.append({"date": date, "equity": ticker, "price": price})

    # Convert the list of dictionaries to a Polars DataFrame
    data = pl.DataFrame(price_records)

    # Define the base directory for news data
    news_data_base_path = '/content/drive/My Drive/NCKH/news_data'
    temp_dir = os.path.join(news_data_base_path, "temp")
    primary_dir = os.path.join(news_data_base_path, "03_primary")

    # Prepare directories
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir, exist_ok=True) # Use exist_ok=True to avoid error if parent dirs don't exist

    if not os.path.exists(primary_dir):
        os.makedirs(primary_dir, exist_ok=True) # Ensure primary directory exists


    # Extract unique (date, equity) pairs
    query_data = (
        data.select(["date", "equity"])
        .unique()
        .to_dict()
    )


    args_list = list(zip(query_data["date"], query_data["equity"]))
    with tqdm(total=len(args_list)) as pbar:
        for i, arg in enumerate(args_list):
            try:
                # Pass the temp directory and include_content=True to query_one_record
                query_one_record(arg, temp_dir, include_content=True, exclude_contentless= True)
            except tenacity.RetryError as e:
                print(f"We caught the error: {e}")
            pbar.update(1)
            if (i + 1) % 3000 == 0:
                time.sleep(90)

    # Combine temporary files
    record_dfs = [
        pl.read_parquet(os.path.join(temp_dir, f))
        for f in os.listdir(temp_dir) if f.endswith(".parquet") # Only read parquet files
    ]

    if record_dfs: # Check if there are any dataframes to concatenate
        df = pl.concat(record_dfs)
        df.write_parquet(os.path.join(primary_dir, "news.parquet"))
        print(f"Combined news data saved to: {os.path.join(primary_dir, 'news.parquet')}")
        print(df.shape)
    else:
        print("No news data was successfully downloaded and processed.")

In [ ]:
if __name__ == "__main__":
    main_sync()

 38%|███▊      | 636/1652 [02:11<03:21,  5.04it/s]

Hit limit

 76%|███████▋  | 1261/1652 [05:19<01:14,  5.27it/s]

Hit limit

100%|██████████| 1652/1652 [07:37<00:00,  3.61it/s]


Combined news data saved to: /content/drive/My Drive/NCKH/news_data/03_primary/news.parquet

(14748, 9)

In [ ]:
import polars as pl
import os

news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news.parquet'

if os.path.exists(news_file_path):
    news_df = pl.read_parquet(news_file_path)
    print("News DataFrame loaded successfully.")
else:
    print(f"News file not found at: {news_file_path}")

News DataFrame loaded successfully.

In [ ]:
print(news_df[:2])

shape: (2, 9)
┌────────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬────────┐
│ author     ┆ content    ┆ datetime   ┆ source   ┆ … ┆ title     ┆ url       ┆ date      ┆ equity │
│ ---        ┆ ---        ┆ ---        ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---    │
│ str        ┆ str        ┆ datetime[μ ┆ str      ┆   ┆ str       ┆ str       ┆ datetime[ ┆ str    │
│            ┆            ┆ s]         ┆          ┆   ┆           ┆           ┆ μs]       ┆        │
╞════════════╪════════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪════════╡
│ Piero      ┆ <p>In a    ┆ 2024-06-14 ┆ benzinga ┆ … ┆ Inflation ┆ https://w ┆ 2024-06-1 ┆ MSFT   │
│ Cingari    ┆ week       ┆ 20:06:26   ┆          ┆   ┆ Cools In  ┆ ww.benzin ┆ 5         ┆        │
│            ┆ filled     ┆            ┆          ┆   ┆ May, Fed  ┆ ga.com/ma ┆ 20:06:26  ┆        │
│            ┆ with       ┆            ┆          ┆   ┆ Hi…       ┆ rke…      ┆           ┆        │
│            ┆ marke…     ┆            ┆          ┆   ┆           ┆           ┆           ┆        │
│ Franca     ┆ <p>Starfie ┆ 2024-06-14 ┆ benzinga ┆ … ┆ Starfield ┆ https://w ┆ 2024-06-1 ┆ MSFT   │
│ Quarneti   ┆ ld, <stron ┆ 19:59:34   ┆          ┆   ┆ Review    ┆ ww.benzin ┆ 5         ┆        │
│            ┆ g>Microsof ┆            ┆          ┆   ┆ Bomb:     ┆ ga.com/ge ┆ 19:59:34  ┆        │
│            ┆ …          ┆            ┆          ┆   ┆ Bethesd…  ┆ ner…      ┆           ┆        │
└────────────┴────────────┴────────────┴──────────┴───┴───────────┴───────────┴───────────┴────────┘

In [ ]:
print(news_df[0]['summary'])

shape: (1,)
Series: 'summary' [str]
[
        "Wall Street resilient as tech …
]

In [ ]:
print(news_df[0, 'summary'])

The Teamsters union on Tuesday announced that it ratified a landmark five-year collective bargaining agreement with
the United Parcel Service.

# 3. Aggregate Reuters news data with Alpaca

The Reuters crawling process is executed in another place. At this phase, i want to aggregate it into the fetched Alpaca news database.

In [ ]:
news_df = news_df.with_columns(
    pl.col("date").cast(pl.Date)
)

print(news_df["date"])

shape: (14_748,)
Series: 'date' [date]
[
        2024-06-15
        2024-06-15
        2024-06-15
        2024-06-15
        2024-06-15
        …
        2024-02-24
        2024-02-24
        2024-02-23
        2024-02-23
        2024-02-23
]

In [ ]:
import polars as pl
import os

news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news.parquet'
sorted_news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news_sorted.parquet'


if os.path.exists(news_file_path):
    print(f"Loading news data from: {news_file_path}")
    news_df = pl.read_parquet(news_file_path)

    print("Sorting news data by 'datetime' in ascending order...")
    sorted_news_df = news_df.sort("datetime", descending=False)

    print(f"Saving sorted news data to a new file: {sorted_news_file_path}")
    sorted_news_df.write_parquet(sorted_news_file_path)

    print("Sorted news data saved successfully.")
else:
    print(f"News file not found at: {news_file_path}")

Loading news data from: /content/drive/My Drive/NCKH/news_data/03_primary/news.parquet
Sorting news data by 'datetime' in ascending order...
Saving sorted news data to a new file: /content/drive/My Drive/NCKH/news_data/03_primary/news_sorted.parquet
Sorted news data saved successfully.


In [ ]:
if os.path.exists(sorted_news_file_path):
    news_df = pl.read_parquet(sorted_news_file_path)
    print(news_df[:20])

shape: (20, 9)
┌────────────┬─────────┬────────────┬──────────┬───┬────────────┬────────────┬────────────┬────────┐
│ author     ┆ content ┆ datetime   ┆ source   ┆ … ┆ title      ┆ url        ┆ date       ┆ equity │
│ ---        ┆ ---     ┆ ---        ┆ ---      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---    │
│ str        ┆ str     ┆ datetime[μ ┆ str      ┆   ┆ str        ┆ str        ┆ datetime[μ ┆ str    │
│            ┆         ┆ s]         ┆          ┆   ┆            ┆            ┆ s]         ┆        │
╞════════════╪═════════╪════════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════╡
│ Jayson     ┆         ┆ 2019-09-17 ┆ benzinga ┆ … ┆ Analyst On ┆ https://ww ┆ 2019-09-18 ┆ MSFT   │
│ Derrick    ┆         ┆ 17:26:14   ┆          ┆   ┆ WeWork:    ┆ w.benzinga ┆ 09:00:00   ┆        │
│            ┆         ┆            ┆          ┆   ┆ High       ┆ .com/analy ┆            ┆        │
│            ┆         ┆            ┆          ┆   ┆ Growth…    ┆ …         

## a. Fixing Datetime error.

This show us that there are **223** instance that have `"Null"` value in column `Date`.

In [ ]:
for col in news_df.columns:
  print(f"column: {col}, ", news_df[col].is_null().sum())

column: author,  0

column: content,  0

column: datetime,  0

column: source,  0

column: summary,  0

...

column: url,  0

column: date,  0

column: equity,  0

column: year,  0

column: month,  0

This possibly because of the Error happens when we transform the original datetime to the roll-over datetime. The original logic in `round_to_next_day(`) functions show flaw in the instance that have "datetime" at the end of the month. So the "+" condition makes the date value falls out of appropriate date range -> Null when we apply `datetime()`.

In [ ]:
null_instances = news_df.filter(pl.col("date").is_null())
null_instances[['datetime','date']]

datetime,date
datetime[μs],date


Mapping the `"Date"` attribute again.

In [ ]:
import polars as pl
import os

sorted_news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news_sorted.parquet'

if os.path.exists(sorted_news_file_path):
    print(f"Loading sorted news data from: {sorted_news_file_path}")
    news_df = pl.read_parquet(sorted_news_file_path)

    print("Recalculating 'date' column to fix nulls...")

    # Apply the condition and use offset_by to add a day where needed
    news_df = news_df.with_columns(
        pl.when(
            (pl.col("datetime").dt.hour() >= 16) & ((pl.col("datetime").dt.minute() > 0) | (pl.col("datetime").dt.second() > 0))
        )
        .then(pl.col("datetime").dt.offset_by("1d").cast(pl.Date))
        .otherwise(pl.col("datetime").cast(pl.Date))
        .alias("date")
    )

    print(f"Saving corrected news data back to: {sorted_news_file_path}")
    news_df.write_parquet(sorted_news_file_path)

    print("Corrected news data saved successfully.")

    # Verify that the nulls in 'date' are gone
    null_count_after_fix = news_df["date"].is_null().sum()
    print(f"Number of nulls in 'date' column after fix: {null_count_after_fix}")

else:
    print(f"Sorted news file not found at: {sorted_news_file_path}")

Loading sorted news data from: /content/drive/My Drive/NCKH/news_data/03_primary/news_sorted.parquet
Recalculating 'date' column to fix nulls...
Saving corrected news data back to: /content/drive/My Drive/NCKH/news_data/03_primary/news_sorted.parquet
Corrected news data saved successfully.
Number of nulls in 'date' column after fix: 0


Tracking the number of News for each tickets

In [ ]:
tickets = ["TSLA", "AMZN", "MSFT", "NFLX"]
total_news = [len(news_df.filter(pl.col("equity") == ticket)) for ticket in tickets]
for ticket, total_new in zip(tickets, total_news):
  print(ticket, total_new)

TSLA 6220

AMZN 3448

MSFT 4070

NFLX 1010

In [ ]:
import plotly.express as px

# Extract year and month for grouping
news_df = news_df.with_columns(
    pl.col("date").dt.year().alias("year"),
    pl.col("date").dt.month().alias("month")
)

# Group by equity, year, and month and count articles
monthly_counts = news_df.group_by(["equity", "year", "month"]).agg(
    pl.count().alias("article_count")
).sort(["equity", "year", "month"])

# Convert month and year to string for better display in the chart
monthly_counts = monthly_counts.with_columns(
    pl.col("year").cast(pl.Utf8),
    pl.col("month").cast(pl.Utf8)
)

# Create a combined month-year string for the x-axis
monthly_counts = monthly_counts.with_columns(
    (pl.col("year") + "-" + pl.col("month")).alias("year_month")
)


# Convert Polars DataFrame to Pandas DataFrame for Plotly
monthly_counts_pd = monthly_counts.to_pandas()

# Create a bar chart
fig = px.bar(monthly_counts_pd,
             x="year_month",
             y="article_count",
             color="equity",
             title="Number of News Articles by Ticker and Month",
             labels={"year_month": "Year-Month", "article_count": "Number of Articles", "equity": "Ticker"})

fig.update_layout(xaxis_tickangle=-45)
fig.show()

/tmp/ipython-input-24-514848126.py:11: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



## b. Aggregate Reuter data


In [ ]:
import pandas as pd

In [ ]:
reuter_news_file = '/content/drive/My Drive/NCKH/news_data/03_primary/Reuters.xlsx'
df = pd.read_excel(reuter_news_file)

In [ ]:
import polars as pl

reuters_df = pl.from_pandas(df)
print(reuters_df.head())

shape: (5, 4)
┌───────────────────────────────┬─────────────────────┬───────────────────────────────┬────────────┐
│ Title                         ┆ datetime_utc        ┆ Content                       ┆ Stock_Type │
│ ---                           ┆ ---                 ┆ ---                           ┆ ---        │
│ str                           ┆ datetime[ns]        ┆ str                           ┆ str        │
╞═══════════════════════════════╪═════════════════════╪═══════════════════════════════╪════════════╡
│ Canada, inspired by EU and    ┆ 2024-06-24 20:22:00 ┆ OTTAWA, June 24 (Reuters) -   ┆ Tesla      │
│ US,…                          ┆                     ┆ Ca…                           ┆            │
│ Delaware law to allow big     ┆ 2024-06-25 18:53:00 ┆ WILMINGTON, Delaware, June 25 ┆ Tesla      │
│ inve…                         ┆                     ┆ …                             ┆            │
│ Electric-vehicle maker Rivian ┆ 2024-06-25 19:39:00 ┆ NORMAL, Illinois, Jun

Rename the columns of the reuters_df Polars DataFrame to match the Alpaca news DataFrame schema and print the schema to verify.



In [ ]:
reuters_df = reuters_df.rename({
    'Title': 'title',
    'datetime_utc': 'datetime',
    'Content': 'content',
    'Stock_Type': 'equity'
})

print(reuters_df.schema)

Schema([('title', String), ('datetime', Datetime(time_unit='ns', time_zone=None)), ('content', String), ('equity', String)])


In [ ]:
reuters_df = reuters_df.with_columns(
    pl.col("datetime").cast(pl.Date).alias("date")
)

print(reuters_df[['datetime', 'date']].head())

shape: (5, 2)
┌─────────────────────┬────────────┐
│ datetime            ┆ date       │
│ ---                 ┆ ---        │
│ datetime[ns]        ┆ date       │
╞═════════════════════╪════════════╡
│ 2024-06-24 20:22:00 ┆ 2024-06-24 │
│ 2024-06-25 18:53:00 ┆ 2024-06-25 │
│ 2024-06-25 19:39:00 ┆ 2024-06-25 │
│ 2024-06-26 05:38:00 ┆ 2024-06-26 │
│ 2024-06-26 13:33:00 ┆ 2024-06-26 │
└─────────────────────┴────────────┘


Select and reorder the columns in the Reuters DataFrame to match the schema of the Alpaca news DataFrame.



In [ ]:
# Define the desired column order
desired_columns = ['title', 'datetime', 'content', 'equity', 'date']

# Select and reorder the columns
reuters_df = reuters_df.select(desired_columns)

# Print the schema to verify
print(reuters_df.schema)

Schema([('title', String), ('datetime', Datetime(time_unit='ns', time_zone=None)), ('content', String), ('equity', String), ('date', Date)])


In [ ]:
news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news.parquet'

if os.path.exists(news_file_path):
    news_df = pl.read_parquet(news_file_path)
    print("News DataFrame loaded successfully.")
else:
    print(f"News file not found at: {news_file_path}")

News DataFrame loaded successfully.


In [ ]:
print("Schema of news_df:")
print(news_df.schema)
print("\nSchema of reuters_df:")
print(reuters_df.schema)

Schema of news_df:
Schema([('author', String), ('content', String), ('datetime', Datetime(time_unit='us', time_zone=None)), ('source', String), ('summary', String), ('title', String), ('url', String), ('date', Datetime(time_unit='us', time_zone=None)), ('equity', String)])

Schema of reuters_df:
Schema([('title', String), ('datetime', Datetime(time_unit='ns', time_zone=None)), ('content', String), ('equity', String), ('date', Date)])


**Reasoning**:
The schemas show that `news_df` has columns 'author', 'source', and 'summary' which are missing in `reuters_df`, and the 'date' column has different types. To make the schemas compatible for concatenation, I need to add the missing columns to `reuters_df` with null values and the correct data types, and ensure the 'date' column in `reuters_df` is of the same type as in `news_df`.



In [ ]:
reuters_df = reuters_df.with_columns([
    pl.lit(None, dtype=pl.String).alias("author"),
    pl.lit(None, dtype=pl.String).alias("source"),
    pl.lit(None, dtype=pl.String).alias("summary"),
    pl.lit(None, dtype=pl.String).alias("url") # Add the missing 'url' column
])

# Cast the 'datetime' column in reuters_df to match the time unit in news_df
reuters_df = reuters_df.with_columns(
    pl.col("datetime").cast(pl.Datetime(time_unit='us')).alias("datetime")
)


# Cast the 'date' column in reuters_df to match the type in news_df
reuters_df = reuters_df.with_columns(
    pl.col("date").cast(pl.Datetime(time_unit='us')).alias("date")
)


# Reorder columns of reuters_df to match news_df
desired_columns = ['author', 'content', 'datetime', 'source', 'summary', 'title', 'url', 'date', 'equity']
reuters_df = reuters_df.select(desired_columns)

print("Schema of reuters_df after modification:")
print(reuters_df.schema)

Schema of reuters_df after modification:
Schema([('author', String), ('content', String), ('datetime', Datetime(time_unit='us', time_zone=None)), ('source', String), ('summary', String), ('title', String), ('url', String), ('date', Datetime(time_unit='us', time_zone=None)), ('equity', String)])


**Reasoning**:
The schema of `reuters_df` now matches the schema of `news_df`. I can now concatenate the two DataFrames, sort the combined DataFrame by date, and save it back to the parquet file as per the original instruction.



In [ ]:
combined_df = pl.concat([news_df, reuters_df])

sorted_combined_df = combined_df.sort("date", descending=False)

news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news.parquet'
sorted_combined_df.write_parquet(news_file_path)

print(f"Combined and sorted news data saved to: {news_file_path}")
print(sorted_combined_df.shape)

Combined and sorted news data saved to: /content/drive/My Drive/NCKH/news_data/03_primary/news.parquet
(24338, 9)


##3. Verify the result




In [ ]:
news_file_path = '/content/drive/My Drive/NCKH/news_data/03_primary/news.parquet'

# Load the combined and sorted parquet file
combined_sorted_news_df = pl.read_parquet(news_file_path)

# Display the first few rows
print("First 5 rows of the combined and sorted news DataFrame:")
print(combined_sorted_news_df.head())

# Display the schema
print("\nSchema of the combined and sorted news DataFrame:")
print(combined_sorted_news_df.schema)

First 5 rows of the combined and sorted news DataFrame:
shape: (5, 9)
┌────────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬────────┐
│ author     ┆ content    ┆ datetime   ┆ source   ┆ … ┆ title     ┆ url       ┆ date      ┆ equity │
│ ---        ┆ ---        ┆ ---        ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---    │
│ str        ┆ str        ┆ datetime[μ ┆ str      ┆   ┆ str       ┆ str       ┆ datetime[ ┆ str    │
│            ┆            ┆ s]         ┆          ┆   ┆           ┆           ┆ μs]       ┆        │
╞════════════╪════════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪════════╡
│ Jayson     ┆ WeWork is  ┆ 2019-09-17 ┆ benzinga ┆ … ┆ Analyst   ┆ https://w ┆ 2019-09-1 ┆ MSFT   │
│ Derrick    ┆ postponing ┆ 17:26:14   ┆          ┆   ┆ On        ┆ ww.benzin ┆ 8         ┆        │
│            ┆ its initi… ┆            ┆          ┆   ┆ WeWork:   ┆ ga.com/an ┆ 17:26:14  ┆        │
│            ┆       

# 4. Process the Reuter datetime to match trading date.

Reuter news datetime need to be align with the trading date in price data. Therefore, we need to map the datetime to the nearest trading date.

In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')
price_data_path = '/content/drive/My Drive/NCKH/price_data/price_data.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
price_df = pd.read_pickle(price_data_path)
print(price_df.keys())

dict_keys([datetime.date(2023, 8, 17), datetime.date(2023, 8, 18), datetime.date(2023, 8, 21), datetime.date(2023, 8, 22), datetime.date(2023, 8, 23), datetime.date(2023, 8, 24), datetime.date(2023, 8, 25), datetime.date(2023, 8, 28), datetime.date(2023, 8, 29), datetime.date(2023, 8, 30), datetime.date(2023, 8, 31), datetime.date(2023, 9, 1), datetime.date(2023, 9, 5), datetime.date(2023, 9, 6), datetime.date(2023, 9, 7), datetime.date(2023, 9, 8), datetime.date(2023, 9, 11), datetime.date(2023, 9, 12), datetime.date(2023, 9, 13), datetime.date(2023, 9, 14), datetime.date(2023, 9, 15), datetime.date(2023, 9, 18), datetime.date(2023, 9, 19), datetime.date(2023, 9, 20), datetime.date(2023, 9, 21), datetime.date(2023, 9, 22), datetime.date(2023, 9, 25), datetime.date(2023, 9, 26), datetime.date(2023, 9, 27), datetime.date(2023, 9, 28), datetime.date(2023, 9, 29), datetime.date(2023, 10, 2), datetime.date(2023, 10, 3), datetime.date(2023, 10, 4), datetime.date(2023, 10, 5), datetime.date(

In [ ]:
# Convert the dictionary to a pandas DataFrame
price_df = pd.DataFrame.from_dict(price_df, orient='index')
price_df.index.name = 'date'
price_df = price_df.reset_index()

print(price_df.head())

         date                                              price
0  2023-08-17  {'TSLA': 219.22000122070312, 'AMZN': 133.97999...
1  2023-08-18  {'TSLA': 215.49000549316406, 'AMZN': 133.22000...
2  2023-08-21  {'TSLA': 231.27999877929688, 'AMZN': 134.67999...
3  2023-08-22  {'TSLA': 233.19000244140625, 'AMZN': 134.25, '...
4  2023-08-23  {'TSLA': 236.86000061035156, 'AMZN': 135.52000...


In [ ]:
from datetime import timedelta
def adjust_trading_days(df: pd.DataFrame, trading_dates: list) -> pd.DataFrame:
    """
    Adjusts dates in the DataFrame to the nearest following trading day,
    using a pre-fetched list of trading dates.

    Parameters:
        df (pd.DataFrame): The DataFrame with a 'date' column to be adjusted.
        trading_dates (list or set): List or set of valid trading datetime.date objects.

    Returns:
        pd.DataFrame: A DataFrame with adjusted 'date' column.
    """
    # Ensure 'date' column is datetime and normalized (no time part)
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

    # Convert trading_dates to a sorted list of datetime.date for iteration
    trading_dates = sorted(pd.to_datetime(trading_dates).normalize())

    # For faster lookup
    trading_set = set(trading_dates)

    # Adjust dates
    adjusted_dates = []
    for current_date in df['date']:
        adjusted_date = current_date
        while adjusted_date not in trading_set:
            adjusted_date += timedelta(days=1)
        adjusted_dates.append(adjusted_date)

    df['date'] = adjusted_dates
    return df

In [ ]:
#Running this is possible, but the nested loop makes it extremely time-consuming.
# df_raw_adjusted = adjust_trading_days(news_raw, trading_date)

In [ ]:
from tqdm import tqdm
import pandas as pd

def build_trading_date_map(trading_dates, start_date, end_date):
    """
    Builds a mapping from each calendar date in the range to the next trading day.
    Uses tqdm for progress visualization.

    Parameters:
        trading_dates (list or Series): A list of valid trading dates.
        start_date (datetime): Start of calendar date range.
        end_date (datetime): End of calendar date range.

    Returns:
        dict: A mapping from each calendar date to the next trading day.
    """
    trading_dates = pd.to_datetime(sorted(pd.to_datetime(trading_dates).normalize()))
    calendar_dates = pd.date_range(start=start_date, end=end_date, freq='D')

    mapping = {}
    trading_idx = 0

    for date in tqdm(calendar_dates, desc="Building trading date map"):
        while trading_idx < len(trading_dates) and trading_dates[trading_idx] < date:
            trading_idx += 1
        if trading_idx < len(trading_dates):
            mapping[date] = trading_dates[trading_idx]
        else:
            mapping[date] = trading_dates[-1]  # fallback if out of range

    return mapping


In [ ]:
from tqdm import tqdm

def adjust_trading_days_fast(df, trading_date_map):
    """
    Adjusts dates in a DataFrame to the next trading day using a precomputed map.
    Shows progress using tqdm.

    Parameters:
        df (pd.DataFrame): The DataFrame with a 'date' column.
        trading_date_map (dict): Mapping from raw dates to trading dates.

    Returns:
        pd.DataFrame: Updated DataFrame with adjusted 'date' column.
    """
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    tqdm.pandas(desc="Adjusting to trading days")
    df['date'] = df['date'].progress_map(lambda x: trading_date_map.get(x, x))
    return df


In [ ]:
news_data_path = '/content/drive/My Drive/NCKH/news_data/concatenated_news_filtered.parquet'
news_data = pd.read_parquet(news_data_path)
news_data['date'] = pd.to_datetime(news_data['date']).dt.normalize()

In [ ]:
trading_date = price_df['date'].tolist()
news_raw = news_data[~news_data['date'].isin(trading_date)] #4468
news_processed = news_data[news_data['date'].isin(trading_date)] #19832

In [ ]:
start_date = news_data['date'].min()
end_date = news_data['date'].max()

# Build the fast lookup mapping
trading_date_map = build_trading_date_map(trading_date, start_date, end_date)

# Adjust in ~O(n) with mapping
df_adjusted = adjust_trading_days_fast(news_raw, trading_date_map)

Building trading date map: 100%|██████████| 2110/2110 [00:00<00:00, 89556.58it/s]
/tmp/ipython-input-37-3016348647.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date']).dt.normalize()
Adjusting to trading days: 100%|██████████| 4468/4468 [00:00<00:00, 285559.84it/s]
/tmp/ipython-input-37-3016348647.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].progress_map(lambda x: trading_date_map.get(x, x))


In [ ]:
df_clean = pd.concat([news_processed, df_adjusted]).sort_values(by='date').reset_index(drop=True)


In [ ]:
print(len(df_clean[~df_clean['date'].isin(trading_date)]))

0


In [ ]:
# Define the path to save the processed data
processed_news_path = '/content/drive/My Drive/NCKH/news_data/concatenated_news_filtered.parquet'

# Save the df_clean DataFrame to the specified path, overwriting the existing file
df_clean.to_parquet(processed_news_path, index=False)

print(f"Processed and cleaned news data saved to: {processed_news_path}")
print(df_clean.shape)

Processed and cleaned news data saved to: /content/drive/My Drive/NCKH/news_data/concatenated_news_filtered.parquet
(24300, 13)
